In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()


In [ ]:
#import 2020 QB stats CSV
DF2020 = pd.read_csv('supp/2020QB.csv')
#splitting on '\' to keep player name
Name = DF2020['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2020['Player Name'] = Name
DF2020['Pos'] = DF2020['Pos'].str.upper()
DF2020 = DF2020.iloc[:, 2:]
DF2020['Year'] = '2020'



In [ ]:
#import 2019 QB stats CSV
DF2019 = pd.read_csv('supp/2019QB.csv')
#splitting on '\' to keep player name
Name = DF2019['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2019['Player Name'] = Name
DF2019['Pos'] = DF2019['Pos'].str.upper()
DF2019 = DF2019.iloc[:, 2:]
DF2019['Year'] = '2019'



In [ ]:
#import 2018 QB stats CSV
DF2018 = pd.read_csv('supp/2019QB.csv')
#splitting on '\' to keep player name
Name = DF2018['Player'].str.split('\\').str[0]
Name = Name.values.tolist()

#function to remove puncutaion from player names
def remove_punc(string):
    punc = '*+'
    for ele in string:
        if ele in punc:
            string = string.replace(ele,'')
    return string
#needed to add .strip() to lagging white space which affected merge downstream
Name = [remove_punc(i).strip() for i in Name]
DF2018['Player Name'] = Name
DF2018['Pos'] = DF2018['Pos'].str.upper()
DF2018 = DF2018.iloc[:, 2:]
DF2018['Year'] = '2018'

In [ ]:
#concat DF here with fields I want
DF_QB = pd.concat([DF2020,DF2019,DF2018])
DF_QB = DF_QB[['Player Name', 'Age', 'Pos', 'Tm', 'Year', 'G', 'GS', 'Cmp', 'Att', 'Yds', 'TD']]
DF_QB.head(5)


In [ ]:
#Loading Player Measureables data
DFPM = pd.read_csv('supp/playermeasureables - Sheet1.csv')
Last_NamePM = DFPM['Player'].str.split(', ').str[0]
Last_NamePM = Last_NamePM.values.tolist()
First_NamePM = DFPM['Player'].str.split(', ').str[1]
First_NamePM = First_NamePM.values.tolist()
DFPM['First Name'] = First_NamePM
DFPM['Last Name'] = Last_NamePM
DFPM['Player Name'] = DFPM[['First Name', 'Last Name']].apply(lambda x: ' '.join(x), axis=1)
DFPM_Info = DFPM[['Player Name', 'Pos', 'Team', 'Height', 'Weight', 'DOB']]
#DFPM_Info.loc[DFPM_Info['Player Name']=='Patrick Mahomes']



In [138]:
#DF_QB.merge(DFPM_Info, how='left', on='Player Name')
DF_QB_PM = pd.merge(DF_QB, DFPM_Info, how='left', on=['Player Name', 'Pos'])
DF_QB_PM = DF_QB_PM[DF_QB_PM['Att'] > 60]
DF_QB_PM.head(5)


,Player Name,Age,Pos,Tm,Year,G,GS,Cmp,Att,Yds,TD,Team,Height,Weight,DOB
0,Matt Ryan,35,QB,ATL,2020,16,16,407,626,4581,26,ATL,"6'4""",217 lbs,5/17/85
1,Tom Brady,43,QB,TAM,2020,16,16,401,610,4633,40,TB,"6'4""",225 lbs,8/3/77
2,Ben Roethlisberger,38,QB,PIT,2020,15,15,399,608,3803,33,PIT,"6'5""",240 lbs,3/2/82
3,Justin Herbert,22,QB,LAC,2020,15,15,396,595,4336,31,LAC,"6'6""",237 lbs,3/10/98
4,Patrick Mahomes,25,QB,KAN,2020,15,15,390,588,4740,38,KC,"6'3""",230 lbs,9/17/95


In [139]:
#convert Height to inches
DF_QB_PM.dtypes
# Feet = DF_QB_PM['Height'].str.split("'").str[0]
# Inches = DF_QB_PM['Height'].str.split("'").str[1].replace("\"","")
# Feet = Feet.values.tolist()
# Inches = Inches.values.tolist()

# def convert_height()

#Height1 = + float(Inches) for value in DF_QB_PM
# def parse_ht(ht):
#     # format: 7' 0.0"
#     ht_ = ht.split("'")
#     ft_ = float(ht_[0])
#     in_ = float(ht_[1].replace("\"",""))
#     return (12*ft_) + in_

# print DF_QB_PM["Height"].apply(lambda x:parse_ht(x))

Player Name    object
Age             int64
Pos            object
Tm             object
Year           object
G               int64
GS              int64
Cmp             int64
Att             int64
Yds             int64
TD              int64
Team           object
Height         object
Weight         object
DOB            object
dtype: object

In [ ]:
# #2018 merged data
# DF_QB_PM_2018 = DF_QB_PM[DF_QB_PM['Year'] == '2018']

# #2019 merged data
# DF_QB_PM_2019 = DF_QB_PM[DF_QB_PM['Year'] == '2019']
# #2020 merged data
# DF_QB_PM_2020 = DF_QB_PM[DF_QB_PM['Year'] == '2020']
# DF_QB_PM_2018.head(30)


In [ ]:
# fig, axes = plt.subplots(1, 3, figsize=(20,10))
# fig.suptitle('Completion Distribution by Height')
# axes[0].set_title('2018')
# axes[1].set_title('2019')
# axes[2].set_title('2020')
#2018
plt.figure(figsize=(8,8))
sns.displot(DF_QB_PM_2018['Height'])
# sns.histplot(ax=axes[0], x=DF_QB_PM[DF_QB_PM['Height']], y=DF_QB_PM[DF_QB_PM['Cmp']]
# axes[0].set_title('2018')
